# Libraries Import

In [ ]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)

import re

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
%matplotlib inline

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Data Import and Preprocessing

In [ ]:
districts = pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv')
products = pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv')

**DISTRICTS**

The districts file includes information about the characteristics of school districts, including data from NCES (2018-19), FCC (Dec 2018), and Edunomics Lab:
1. district_id : Unique Identifier for each district
2. state : The state the district falls in
3. locale : NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural.
4. pct_black/hispanic - percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data.
5. pct_free/reduced - percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data.
6. county_connections_ratio - ratio (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version).
7. pp_total_raw - per-pupil total expenditure (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project.



In [ ]:
districts.head()

In [ ]:
districts.info()

The districts info shows some null values

In [ ]:
# Getting percentage of null values in the dataset
print(f'The district data has {round(sum(districts.isna().sum()) / (districts.shape[0] * districts.shape[1]) * 100, 2)} % missing values')

In [ ]:
#Dropping Districts with NaN States
districts.dropna(subset=['state'],inplace = True)

**PRODUCTS**

In [ ]:
products.head()

In [ ]:
products.info()

Products dataset has some null values

One-Hot Encoding the Product Sectors:

In [ ]:
temp_sectors = products['Sector(s)'].str.get_dummies(sep="; ")
temp_sectors.columns = [f"sector_{re.sub(' ', '', c)}" for c in temp_sectors.columns]
products = products.join(temp_sectors)
products.drop("Sector(s)", axis=1, inplace=True)

del temp_sectors

Splitting the primary essential functions

In [ ]:
products['primary_function_main'] = products['Primary Essential Function'].apply(lambda x: x.split(' - ')[0] if x == x else x)
products['primary_function_sub'] = products['Primary Essential Function'].apply(lambda x: x.split(' - ')[1] if x == x else x)

# Synchronize similar values
products['primary_function_sub'] = products['primary_function_sub'].replace({'Sites, Resources & References' : 'Sites, Resources & Reference'})
products.drop("Primary Essential Function", axis=1, inplace=True)

In [ ]:
path =  '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data'

files = []

for district in districts.district_id.unique():
    df = pd.read_csv(f'{path}/{district}.csv', index_col=None, header=0)
    df["district_id"] = district
    files.append(df)
    
# To merge each districts engagement data into one dataframe
engagement = pd.concat(files, ignore_index=True)

del files
# To convert time column type to datetime
engagement['time'] = pd.to_datetime(engagement['time'])
engagement.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4))

sns.histplot(engagement.groupby('district_id').time.nunique(), bins=30)
ax.set_title('Unique Days of Engagement Data per District')
plt.show()

To make the data easier to analyse, we will only consider distrcits with engagement data for everyday in 2020.

In [ ]:
# Delete previously created engagement dataframe and create a new one
del engagement

files = []

for district in districts.district_id.unique():
    df = pd.read_csv(f'{path}/{district}.csv', index_col=None, header=0)
    df["district_id"] = district
    if df.time.nunique() == 366:
        files.append(df)

engagement = pd.concat(files, ignore_index=True)

# Only consider districts with full 2020 engagement data
districts = districts[districts.district_id.isin(engagement.district_id.unique())].reset_index(drop=True)
products = products[products['LP ID'].isin(engagement.lp_id.unique())].reset_index(drop=True)

Furthermore, if we look at a few sample districts, we can  see that most districts use more than the 369 unique products from products_info. In fact, the concatenated engagement data contains more than 8000 unique products. Since we don't have any additional information the majority of these products, we will remove engagement data for unknown products. This reduces the engagement data roughly by half.

In [ ]:
for district in districts.district_id.unique()[:10]:
    df = pd.read_csv(f'{path}/{district}.csv', index_col=None, header=0)
    print(f'District {district} uses {df.lp_id.nunique()} unique products.')
    
print(f'\nConcatenated engagement data contains {engagement.lp_id.nunique()} unique products.')

In [ ]:
print(len(engagement))
engagement = engagement[engagement.lp_id.isin(products['LP ID'].unique())]
print(len(engagement))

To summarize, we have removed districts without any information on the location and we have removed districts with incomplete data in 2020.

Finally, we will convert the time column to the type datetime64[ns] for easier handling.

In [ ]:
engagement.time = engagement.time.astype('datetime64[ns]')

# Exploratory Data Analysis

**DISTRICTS**

In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District Of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

districts['state_abbrev'] = districts['state'].replace(us_state_abbrev)
districts_by_state = districts['state_abbrev'].value_counts().to_frame().reset_index(drop=False)
districts_by_state.columns = ['state_abbrev', 'num_districts']

fig = go.Figure()
layout = dict(
    title_text = "Number of Available School Districts per State",
    geo_scope='usa',
)

fig.add_trace(
    go.Choropleth(
        locations=districts_by_state.state_abbrev,
        zmax=1,
        z = districts_by_state.num_districts,
        locationmode = 'USA-states', # set of locations match entries in `locations`
        marker_line_color='white',
        geo='geo',
        colorscale=px.colors.sequential.Teal, 
    )
)
            
fig.update_layout(layout)   
fig.show()

plt.figure(figsize = (15, 8))
sns.set_style("white")
a = sns.barplot(data = districts['state'].value_counts().reset_index(), x = 'state', y = 'index', color = '#90afc5')
plt.xticks([])
plt.yticks(fontname = 'monospace', fontsize = 14, color = '#283655')
plt.ylabel('')
plt.xlabel('')

a.spines['left'].set_linewidth(1.5)
for w in ['right', 'top', 'bottom']:
    a.spines[w].set_visible(False)
    
for p in a.patches:
    width = p.get_width()
    plt.text(0.5 + width, p.get_y() + 0.55 * p.get_height(), f'{int(width)}',
             ha = 'center', va = 'center', fontname = 'monospace', fontsize = 15, color = '#283655')

plt.show()

In [ ]:
fig = px.pie(districts['locale'].value_counts().reset_index().rename(columns = {'locale': 'count'}), values = 'count', names = 'index', width = 700, height = 700)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.7, 
                  marker = dict(colors = ['#90afc5','#336b87','#2a3132','#763626'], line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = ' The count of districts <br>in each type <br>of areas', 
                                      x = 0.5, y = 0.5, font_size = 26, showarrow = False, 
                                      font_family = 'monospace',
                                      font_color = '#283655')],
                  showlegend = False)
                  
fig.show()

In [ ]:
districts.pp_total_raw.unique()
temp = districts.groupby('locale').pp_total_raw.value_counts().to_frame()
temp.columns = ['amount']

temp = temp.reset_index(drop=False)

temp = temp.pivot(index='locale', columns='pp_total_raw')['amount']
temp = temp[['[4000, 6000[', '[6000, 8000[', '[8000, 10000[', '[10000, 12000[',
       '[12000, 14000[', '[14000, 16000[', '[16000, 18000[', 
       '[18000, 20000[', '[20000, 22000[', '[22000, 24000[', ]]


fig, ax = plt.subplots(1, 2, figsize=(24,4))

sns.countplot(data=districts, x='locale', ax=ax[0], palette='GnBu')
ax[0].set_title('Locale Distribution in Districts Data')
sns.heatmap(temp, annot=True,  cmap='GnBu', ax=ax[1])
ax[1].set_title('Heatmap of Districts According To locale and pp_total_raw')
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(16,8))

sns.countplot(data=districts, x='pct_black/hispanic', order=['[0, 0.2[', '[0.2, 0.4[', '[0.4, 0.6[', '[0.6, 0.8[','[0.8, 1[', ], palette='GnBu', ax=ax[0,0])
ax[0,0].set_ylim([0,135])
sns.countplot(data=districts, x='pct_free/reduced', order=['[0, 0.2[', '[0.2, 0.4[', '[0.4, 0.6[', '[0.6, 0.8[','[0.8, 1[', ], palette='GnBu', ax=ax[0,1])
ax[0,1].set_ylim([0,135])

sns.countplot(data=districts, x='county_connections_ratio', palette='GnBu', ax=ax[1,0])
ax[1,0].set_ylim([0,135])
sns.countplot(data=districts, x='pp_total_raw', order=['[4000, 6000[', '[6000, 8000[', '[8000, 10000[', '[10000, 12000[',
       '[12000, 14000[', '[14000, 16000[', '[16000, 18000[', 
       '[18000, 20000[', '[20000, 22000[', '[22000, 24000[', ], palette='GnBu', ax=ax[1,1])
ax[1,1].set_ylim([0,135])
ax[1,1].set_xticklabels(ax[1,1].get_xticklabels(), rotation=90)

plt.tight_layout()
plt.show()

The plots show the column county_connection_ratio is the same throughout the dataset, so it will be dropped

In [ ]:
districts['pct_free/reduced'] = districts[['pct_free/reduced']].applymap(lambda x:float(x.split(',')[0][1:]) + 0.1, na_action='ignore')

districts['pct_black/hispanic'] = districts['pct_black/hispanic'].apply(lambda x:float(x.split(',')[0][1:]) + 0.1)


districts.head(3)

In [ ]:
districts['pp_total_raw'] = districts[['pp_total_raw']].applymap(lambda x: int(x.split(',')[0][1:]) + 1000,
                                                                 na_action='ignore')

districts.drop('county_connections_ratio', axis = 1, inplace = True)

In [ ]:
districts.head(3)

In [ ]:
def plot_state_mean_for_var(col):
    temp = districts.groupby('state_abbrev')[col].mean().to_frame().reset_index(drop=False)

    fig = go.Figure()
    layout = dict(
        title_text = f"Mean {col} per State",
        geo_scope='usa',
    )

    fig.add_trace(
        go.Choropleth(
            locations=temp.state_abbrev,
            zmax=1,
            z = temp[col],
            locationmode = 'USA-states', # set of locations match entries in `locations`
            marker_line_color='white',
            geo='geo',
            colorscale=px.colors.sequential.Teal, 
        )
    )

    fig.update_layout(layout)   
    fig.show()

plot_state_mean_for_var('pct_black/hispanic')
plot_state_mean_for_var('pct_free/reduced')
plot_state_mean_for_var('pp_total_raw')


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,4))
sns.countplot(data=products, x='primary_function_main', palette ='GnBu', ax=ax[0])
ax[0].set_title('Main Categories in Primary Functions')

sns.countplot(data=products[products.primary_function_main == 'LC'], x='primary_function_sub', palette ='GnBu', ax=ax[1])
ax[1].set_title('Sub-Categories in Primary Function LC')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
plt.show()

The most common category in the column Primary Essential Function is Learning & Curriculum (LC) as shown in the above figure. For the categories Classroom Management (CM) and School & District Operations (SDO) there are far fewer tool options available.

In [ ]:
virtual_classroom_lp_id = products[products.primary_function_sub == 'Virtual Classroom']['LP ID'].unique()

# Remove weekends from the dataframe
engagement['weekday'] = pd.DatetimeIndex(engagement['time']).weekday
engagement_without_weekends = engagement[engagement.weekday < 5]

# Figure 1
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(24, 6))
for virtual_classroom_product in virtual_classroom_lp_id:
    temp = engagement_without_weekends[engagement_without_weekends.lp_id == virtual_classroom_product].groupby('time').pct_access.mean().to_frame().reset_index(drop=False)
    sns.lineplot(x=temp.time, y=temp.pct_access, label=products[products['LP ID'] == virtual_classroom_product]['Product Name'].values[0])
plt.legend()
plt.show()

# Figure 2
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(24, 6))
for virtual_classroom_product in virtual_classroom_lp_id:
    temp = engagement_without_weekends[engagement_without_weekends.lp_id == virtual_classroom_product].groupby('time').engagement_index.mean().to_frame().reset_index(drop=False)
    sns.lineplot(x=temp.time, y=temp.engagement_index, label=products[products['LP ID'] == virtual_classroom_product]['Product Name'].values[0])
plt.title('')
plt.legend()
plt.show()


Over the last quarter of 2020, we can see a pct_access of roughly 15. What does this mean? 15 % of students in the district have at least one page-load event of Zoom or Meet. To be honest, that seems a little bit low from the home schooling point of view given that every student needs to attend classes on a school day. However, this seems to hint at the fact that not all students had to attend classes virtually but were able to physically go to school. Judging from this State-by-State Map of Where School Buildings Are Opened or Closed it seems like a lot of schools offered in-person education in 2020. That means, when looking at digital learning, we should probably focus our analysis on districts where digital learning was actually applied to get some key insights.

While for pct_access Zoom and Meet seem to have roughly similiar values, we can see in the lower graph that Meet has more than 4 times the value of Zoom for engagement_index in the last quarter of 2020. What does this mean? If we have 1000 page-load events per 1000 students for Zoom on a given day that means that one student uses Zoom once a day. In contrast, Meet is used 4 or 5 times daily on average per student.

To make things a little bit clearer: If every students has two applications on their phone, the pct_access indicates how many of the students access this app on a daily basis but the engagement_index tells you how much the students engage with that application on a daily basis.


In [ ]:
products['lp_id'] = products['LP ID'].copy()

f, ax = plt.subplots(nrows=3, ncols=3, figsize=(18, 8))

i = 0
j = 0
for subfunction in products[products.primary_function_main == 'LC'].primary_function_sub.unique():
    lp_ids = products[products.primary_function_sub == subfunction]['LP ID'].unique()

    temp = engagement_without_weekends[engagement_without_weekends.lp_id.isin(lp_ids)]
    temp = temp.groupby('lp_id').pct_access.mean().sort_values(ascending=False).to_frame().reset_index(drop=False)
    temp = temp.merge(products[['lp_id', 'Product Name']], on='lp_id').head()
    
    sns.barplot(data=temp, x='pct_access', y='Product Name', palette='GnBu', ax=ax[i, j])
    
    ax[i, j].set_title(f'Top 5 in \n{subfunction}', fontsize=12)
    ax[i, j].set_xlim([0, 20])
    j = j + 1
    if j == 3:
        i = i + 1
        j = 0
        
f.delaxes(ax[2, 1])
f.delaxes(ax[2, 2])

plt.tight_layout()
plt.show()

Above we can see the top 5 most accessed products for each LC sub-category sorted by the mean pct_access for 2020 over all districts. We can see that most of the products are on average accessed by less than 5 % of students on a daily basis. Exceptions are YouTube, Google Docs, and Canvas. YouTube in this case is a difficult one to evaluate since it can be used for leisure in addition to education, so we need to be careful here. Google Docs seems to make a lot of sense since students can use Google Docs. 

Canvas is a web-based learning management system, or LMS. It is used by learning institutions, educators, and students to access and manage online course learning materials and communicate about skill development and learning achievement, so it also makes sense that this one is quite often accessed.

Regarding the sub-category 'Career Planning & Job Search' the average pct_access is very low. This is probably due top the fact that career planning is only relevant to older students. Therefore, we can probably exclude this subcategory when inspecting the digital learning aspect.

In [ ]:
engagement['quarter'] = pd.DatetimeIndex(engagement['time']).quarter.astype(str)

temp = engagement.merge(products[['lp_id', 'Product Name', 'primary_function_main', 'primary_function_sub']], on='lp_id')
temp = temp[temp.primary_function_sub.notna()]
temp = temp.groupby(['quarter', 'primary_function_sub'])['pct_access', 'engagement_index'].mean().reset_index(drop=False)

temp = temp.pivot(index='primary_function_sub', columns='quarter')[['pct_access', 'engagement_index']].fillna(0)

temp.columns = ["_".join(a) for a in temp.columns.to_flat_index()]

temp['pct_access_delta'] = temp['pct_access_4'] - temp['pct_access_1']
temp['engagement_index_delta'] = temp['engagement_index_4'] - temp['engagement_index_1']
temp=temp.reset_index(drop=False)
#temp = temp.merge(products_info[['lp_id', 'Product Name', 'primary_function_sub']], on='lp_id')

f, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))

df = temp.sort_values(by='pct_access_delta', ascending=False)#.head(5)

sns.barplot(data=df, x='pct_access_delta', y='primary_function_sub', palette='GnBu', ax=ax[0])

df = temp.sort_values(by='engagement_index_delta', ascending=False)#.head(5)

sns.barplot(data=df, x='engagement_index_delta', y='primary_function_sub', palette='GnBu', ax=ax[1])
plt.tight_layout()
plt.show()

In [ ]:
temp = engagement.fillna(0).groupby(['quarter', 'lp_id'])['pct_access', 'engagement_index'].mean().reset_index(drop=False)

temp = temp.pivot(index='lp_id', columns='quarter')[['pct_access', 'engagement_index']].fillna(0)

temp.columns = ["_".join(a) for a in temp.columns.to_flat_index()]

temp['pct_access_delta'] = temp['pct_access_4'] - temp['pct_access_1']
temp['engagement_index_delta'] = temp['engagement_index_4'] - temp['engagement_index_1']
temp = temp.merge(products[['lp_id', 'Product Name', 'primary_function_sub']], on='lp_id')

f, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))

df = temp.sort_values(by='pct_access_delta', ascending=False).head(10)

sns.barplot(data=df, x='pct_access_delta', y='Product Name', palette='GnBu', ax=ax[0])

df = temp.sort_values(by='engagement_index_delta', ascending=False).head(10)

sns.barplot(data=df, x='engagement_index_delta', y='Product Name', palette='GnBu', ax=ax[1])
plt.tight_layout()
plt.show()